In [15]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoConfig, pipeline
import torch.neuron
import numpy as np
import os

from proofreading import make_tokens, insert_mask_to_tokens, replace_mask_to_tokens, make_senetence_from_tokens

In [16]:
max_length = 256
text = "これでいけなかったらもうどうしていいのかわからないよ頼むから通ってくれ"

In [17]:
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
tokenizer2 = AutoTokenizer.from_pretrained("xlm-roberta-base")

In [5]:
tokenizer.mask_token
sample_input = f"これはtrace時に{tokenizer.mask_token}使うためのダミーテキストです"
sample_input

'これはtrace時に[MASK]使うためのダミーテキストです'

In [18]:
model_neuron = torch.jit.load('bert_neuron.pt')

In [19]:
model_neuron

RecursiveScriptModule(
  original_name=AwsNeuronGraphModule
  (_NeuronGraph#82): RecursiveScriptModule(original_name=NeuronModuleV2)
)

In [20]:
corrected_tokens = make_tokens(text=text, tokenizer=tokenizer2)[1:]

In [21]:
masked_text = make_senetence_from_tokens(insert_mask_to_tokens(i=5, mask_token="[MASK]", tokens=corrected_tokens))

In [22]:
masked_text

'これでいけなかったら[MASK]もうどうしていいのかわからないよ頼むから通ってくれ'

In [23]:
inputs = tokenizer(
   text=masked_text,
   return_tensors="pt",
)

In [37]:
sample_input = "これはtrace時に[MASK]使うためのダミーテキストですねこれは激熱"
# 入力の最大系列長を事前に指定する
max_length = 256
inputs = tokenizer(
   text=sample_input,
   return_tensors="pt",
   padding="max_length",
   max_length=max_length,
   truncation=True
)

In [39]:
# inputs['input_ids'].size()
# inputs['attention_mask'].size()
inputs['input_ids']

tensor([[    2,   171,     9, 19551,  2953,    72,     7,     4,  3002,    82,
             5,   314,  1659, 10306,  2992,  1852,   171,     9,  1762,  1583,
             3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [40]:
output = model_neuron(*(inputs['input_ids'],inputs['attention_mask']))

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__/torch_neuron/runtime/___torch_mangle_442.py", line 37, in fallback_function
    _16 = torch.slice(_15, 1, 0, _13)
    _17 = torch.embedding(CONSTANTS.c5, argument_1, 0)
    _18 = torch.add(_17, torch.embedding(CONSTANTS.c6, _4))
          ~~~~~~~~~ <--- HERE
    _19 = torch.add(_18, torch.embedding(CONSTANTS.c7, _16))
    model = _NeuronGraph_82.model

Traceback of TorchScript, original code (most recent call last):
/home/ubuntu/pytorch_venv/lib/python3.7/site-packages/torch_neuron/resolve_function.py(68): func_from_schema
/home/ubuntu/pytorch_venv/lib/python3.7/site-packages/torch_neuron/graph.py(330): __call__
/home/ubuntu/pytorch_venv/lib/python3.7/site-packages/torch_neuron/graph.py(207): run_op
/home/ubuntu/pytorch_venv/lib/python3.7/site-packages/torch_neuron/graph.py(196): __call__
/home/ubuntu/pytorch_venv/lib/pyt

In [29]:
output

(tensor([[[-0.3828, -0.3477],
          [-0.5312, -0.1289],
          [-0.2158, -0.2500],
          [-0.3223, -0.5117],
          [-0.5000, -0.2871],
          [-0.4062, -0.0067],
          [-0.7422, -0.0522],
          [-0.5352, -0.2930],
          [-0.5312, -0.1748],
          [-0.3359, -0.5156],
          [-0.6328, -0.3926],
          [-0.3184, -0.3789],
          [-0.5703, -0.5352],
          [-0.2324, -0.0605],
          [-0.6797, -0.0123],
          [-0.4492, -0.1943]]]),)

In [25]:
# tokens = tokenizer.batch_encode_plus(text,max_length=256,padding='max_length', truncation=True)
tokens = tokenizer.batch_encode_plus(text,max_length=256,padding='max_length')
  
text_seq = torch.tensor(tokens['input_ids'])
text_mask = torch.tensor(tokens['attention_mask'])

In [14]:
print(text_seq.size())
text_seq[0][0:20]

torch.Size([35, 256])


tensor([ 2, 27,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0])

In [21]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [38]:
output = model(**inputs)

In [40]:
logit = torch.argmax(output[0]) 

In [41]:
logit

tensor(4623)

In [28]:
results = torch.softmax(logits, dim=1).tolist()[0]

In [42]:
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(logit))

TypeError: iteration over a 0-d tensor

In [20]:
##以下、tokenizer用の入力形式統一テスト
max_length = 256
sample_input = "これはtrace時に[MASK]使うためのダミーテキストです"

In [24]:
inputs = tokenizer(
   text=sample_input,
   return_tensors="pt",
    padding=True,
)

In [25]:
len(inputs)
print(inputs['input_ids'].size())

torch.Size([1, 16])


In [28]:
text = "これでいけなかったらもうどうしていいのかわからないよ頼むから通ってくれ"

In [32]:
inputs = tokenizer(
   text=text,
   return_tensors="pt",
    padding="max_length",
    max_length=max_length,
)

In [33]:
len(inputs)
print(inputs['input_ids'].size())

torch.Size([1, 256])
